In [ ]:
#!pip install --upgrade gspread_dataframe

In [ ]:
from gspread_dataframe import set_with_dataframe, get_as_dataframe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()


import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

# Try to open the sheet, create it if it doesn't exist
sheet_name = "VR_Archery_Labels"

try:
    sheet = gc.open(sheet_name).sheet1
    print(f"Opened existing sheet: {sheet_name}")
except gspread.SpreadsheetNotFound:
    sheet = gc.create(sheet_name).sheet1
    print(f"Created new sheet: {sheet_name}")

Mounted at /content/drive
Opened existing sheet: VR_Archery_Labels


In [ ]:
# import json
# import re
# import glob
# import pandas as pd

# # --------- ID Extraction ---------
# def get_id(data):
#     return re.sub(r'[^\d]', '', data.get("datetime"))

# # --------- Joint Position Extraction ---------
# def get_joint_positions(data):
#     joint_positions = {}
#     for key, value in data["values"].items():
#         if value["Name"].startswith("joint_"):
#             joint_name = value["Name"]
#             joint_positions[joint_name] = value["Position"]
#     return joint_positions

# # --------- Floor Extraction ---------
# def get_floor_position(data):
#     for key, value in data["values"].items():
#         if value["Name"] == "Terrain_Floor":
#             return value["LocalPosition"]
#     return None

# # --------- Target Extraction ---------
# def get_target_positions(data):
#     target_positions = {}
#     for key, value in data["values"].items():
#         if value["Name"].startswith("Target"):
#             target_name = value["Name"]
#             target_positions[target_name] = value["Position"]
#     return target_positions

# # --------- JSON Reading ---------
# def read_json_data(path):
#     entries = []
#     for line in open(path):
#         entry_data = {}
#         entry = json.loads(line)
#         entry_data['ID'] = get_id(entry)
#         entry_data['Joints'] = get_joint_positions(entry)
#         entry_data['Floor'] = get_floor_position(entry)
#         entry_data['Targets'] = get_target_positions(entry)
#         entries.append(entry_data)
#     return entries

# # --------- Global ID Counter ---------
# global_id_counter = 1
# def get_global_id():
#     global global_id_counter
#     current_id = global_id_counter
#     global_id_counter += 1
#     return current_id

# # --------- DataFrame Conversion ---------
# def data_to_dataframe(data_entries):
#     records = []

#     for entry in data_entries:
#         record = {'GlobalID': get_global_id(), 'EntryID': entry['ID']}

#         # Flatten Joints
#         for joint in entry['Joints']:
#             record[f'{joint}_X'] = entry['Joints'][joint][0]
#             record[f'{joint}_Y'] = entry['Joints'][joint][1]
#             record[f'{joint}_Z'] = entry['Joints'][joint][2]

#         # Flatten Targets
#         for target in entry['Targets']:
#             record[f'{target}_X'] = entry['Targets'][target][0]
#             record[f'{target}_Y'] = entry['Targets'][target][1]
#             record[f'{target}_Z'] = entry['Targets'][target][2]

#         # Floor Position
#         if entry['Floor'] is not None:
#             record['Floor_X'] = entry['Floor'][0]
#             record['Floor_Y'] = entry['Floor'][1]
#             record['Floor_Z'] = entry['Floor'][2]

#         # Placeholder labels
#         for label in ['Spatial', 'Temporal', 'Spatial_1', 'Temporal_1', 'Spatial_2', 'Temporal_2']:
#             record[label] = ''

#         records.append(record)

#     return pd.DataFrame(records, dtype=str)

In [3]:
import glob
import pandas as pd
import numpy as np
import json
import re

# --------- ID Extraction ---------
def get_id(data):
    return re.sub(r'[^\d]', '', data.get("datetime"))

# --------- Joint Position Extraction ---------
def get_joint_positions(data):
    joint_positions = {}
    for key, value in data["values"].items():
        if value["Name"].startswith("joint_"):
            joint_name = value["Name"]
            joint_positions[joint_name] = value["Position"]
    return joint_positions

# --------- Floor Extraction ---------
def get_floor_position(data):
    for key, value in data["values"].items():
        if value["Name"] == "Terrain_Floor":
            return value["Position"]
    return None

# --------- Target Extraction ---------
def get_target_positions(data):
    target_positions = {}
    for key, value in data["values"].items():
        if value["Name"].startswith("Target"):
            target_name = value["Name"]
            target_positions[target_name] = value["Position"]
    return target_positions

# --------- Target Rotation Extraction ---------
def get_target_rotation(data):
    target_positions = {}
    for key, value in data["values"].items():
        if value["Name"].startswith("Target"):
            target_name = value["Name"]
            target_positions[target_name] = value["Rotation"]
    return target_positions

# --------- Arrow Extraction ---------
def get_arrow_position(data):
    for key, value in data["values"].items():
        if value["Name"] == "ArrowPosition-Hand":
            return value["Position"]
    return None

# --------- Arrow Rotation Extraction ---------
def get_arrow_rotation(data):
    for key, value in data["values"].items():
        if value["Name"] == "ArrowPosition-Hand":
            return value["Rotation"]
    return None

# --------- JSON Reading ---------
def read_json_data(path):
    entries = []
    for line in open(path):
        entry_data = {}
        entry = json.loads(line)
        entry_data['ID'] = get_id(entry)
        entry_data['Joints'] = get_joint_positions(entry)
        entry_data['Floor'] = get_floor_position(entry)
        entry_data['Arrow'] = get_arrow_position(entry)
        entry_data['ArrowRotation'] = get_arrow_rotation(entry)
        entry_data['Targets'] = get_target_positions(entry)
        entry_data['TargetsRotation'] = get_target_rotation(entry)
        entries.append(entry_data)
    return entries

# --------- Global ID Counter ---------
global_id_counter = 1

def get_global_id():
    global global_id_counter
    current_id = global_id_counter
    global_id_counter += 1
    return current_id

# --------- DataFrame Conversion ---------
def data_to_dataframe(data_entries):
    records = []

    for entry in data_entries:
        record = {'GlobalID': get_global_id(), 'EntryID': entry['ID']}

        # Flatten Joints
        for joint in entry['Joints']:
            record[f'{joint}_X'] = entry['Joints'][joint][0]
            record[f'{joint}_Y'] = entry['Joints'][joint][1]
            record[f'{joint}_Z'] = entry['Joints'][joint][2]

        # Flatten Targets
        for target in entry['Targets']:
            record[f'{target}_X'] = entry['Targets'][target][0]
            record[f'{target}_Y'] = entry['Targets'][target][1]
            record[f'{target}_Z'] = entry['Targets'][target][2]

        # Floor Position
        if entry['Floor'] is not None:
            record['Floor_X'] = entry['Floor'][0]
            record['Floor_Y'] = entry['Floor'][1]
            record['Floor_Z'] = entry['Floor'][2]

        # Arrow Position
        if entry['Arrow'] is not None:
            record['Arrow_X'] = entry['Arrow'][0]
            record['Arrow_Y'] = entry['Arrow'][1]
            record['Arrow_Z'] = entry['Arrow'][2]

        # Arrow Rotation
        if entry['Arrow'] is not None:
            record['Arrow_RotX'] = entry['ArrowRotation'][0]
            record['Arrow_RotY'] = entry['ArrowRotation'][1]
            record['Arrow_RotZ'] = entry['ArrowRotation'][2]
            record['Arrow_RotW'] = entry['ArrowRotation'][3]

        # Target Rotation
        for target in entry['TargetsRotation']:
            record[f'{target}_RotX'] = entry['TargetsRotation'][target][0]
            record[f'{target}_RotY'] = entry['TargetsRotation'][target][1]
            record[f'{target}_RotZ'] = entry['TargetsRotation'][target][2]
            record[f'{target}_RotW'] = entry['TargetsRotation'][target][3]


        # Placeholder labels
        for label in ['Spatial', 'Temporal', 'Spatial_1', 'Temporal_1', 'Spatial_2', 'Temporal_2']:
            record[label] = ''

        records.append(record)

    return pd.DataFrame(records, dtype=str)

In [4]:
def plot_skeleton_on_ax(row, ax):
    import numpy as np

    # Define bones grouped by limb
    bone_groups = {
        'spine': [
            ('joint_Pelvis', 'joint_HipMaster'),
            ('joint_HipMaster', 'joint_TorsoA'),
            ('joint_TorsoA', 'joint_TorsoB'),
            ('joint_TorsoB', 'joint_TorsoC'),
            ('joint_TorsoC', 'joint_Neck'),
            ('joint_Neck', 'joint_Head'),
        ],
        'left_arm': [
            ('joint_TorsoC', 'joint_ClavicleLT'),
            ('joint_ClavicleLT', 'joint_ShoulderLT'),
            ('joint_ShoulderLT', 'joint_ElbowLT'),
            ('joint_ElbowLT', 'joint_HandLT'),
        ],
        'right_arm': [
            ('joint_TorsoC', 'joint_ClavicleRT'),
            ('joint_ClavicleRT', 'joint_ShoulderRT'),
            ('joint_ShoulderRT', 'joint_ElbowRT'),
            ('joint_ElbowRT', 'joint_HandRT'),
        ],
        'left_leg': [
            ('joint_Pelvis', 'joint_HipLT'),
            ('joint_HipLT', 'joint_KneeLT'),
            ('joint_KneeLT', 'joint_FootLT'),
            ('joint_FootLT', 'joint_ToeLT'),
        ],
        'right_leg': [
            ('joint_Pelvis', 'joint_HipRT'),
            ('joint_HipRT', 'joint_KneeRT'),
            ('joint_KneeRT', 'joint_FootRT'),
            ('joint_FootRT', 'joint_ToeRT'),
        ]
    }

    limb_colors = {
        'spine': 'blue',
        'left_arm': 'cyan',
        'right_arm': 'fuchsia',
        'left_leg': 'red',
        'right_leg': 'lime'
    }

    # Parse joint coordinates
    joint_names = sorted(set(
        col[:-2] for col in row.index
        if col.startswith("joint_") and col.endswith(('_X', '_Y', '_Z')) and not col.startswith("joint_Char")
    ))
    joint_coords = {}
    for name in joint_names:
        x = float(row[f'{name}_X'])
        y = float(row[f'{name}_Y'])
        z = float(row[f'{name}_Z'])
        joint_coords[name] = (x, y, z)

    # Plot joints
    # for coord in joint_coords.values():
    #     ax.scatter(*coord, color='blue', s=10)

    # Identify all joints used in bones
    connected_joints = set(j for bone_list in bone_groups.values() for pair in bone_list for j in pair)

    for name, coord in joint_coords.items():
        if name in connected_joints:
            ax.scatter(*coord, color='blue', s=10)
        else:
            ax.scatter(*coord, color='blue', s=5)  # Smaller

    # Plot bones
    for limb, bone_list in bone_groups.items():
        color = limb_colors[limb]
        for j1, j2 in bone_list:
            if j1 in joint_coords and j2 in joint_coords:
                x1, y1, z1 = joint_coords[j1]
                x2, y2, z2 = joint_coords[j2]
                ax.plot([x1, x2], [y1, y2], [z1, z2], color=color)

    # Draw floor rectangle
    floor_x = float(row['joint_HipMaster_X'])
    floor_y = float(row['joint_Char_Y'])
    floor_z = float(row['joint_HipMaster_Z'])

    floor_w, floor_d = 3.0, 3.0
    x_min, x_max = floor_x - floor_w, floor_x + floor_w
    z_min, z_max = floor_z - floor_d, floor_z + floor_d
    ax.plot([x_min, x_max, x_max, x_min, x_min],
            [floor_y] * 5,
            [z_min, z_min, z_max, z_max, z_min],
            color='purple', alpha=0.6)
    ax.scatter(floor_x, floor_y, floor_z, color='red', s=40, marker='x')

    # Axis labels
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

In [5]:
import os
import matplotlib.pyplot as plt

def save_skeleton_views_and_upload(row, output_dir_drive_path):
    entry_id = row['EntryID']
    view_configs = [('view1', 20, 20), ('view2', -90, 90), ('view3', 0, 20)]
    uploaded_file_ids = []

    # Local Colab path for temporary saving before upload
    local_tmp_dir = "/content/tmp_plots"
    os.makedirs(local_tmp_dir, exist_ok=True)

    for view_name, elev, azim in view_configs:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(elev=elev, azim=azim)
        plot_skeleton_on_ax(row, ax)

        # Save locally
        filename = f"{entry_id}_{view_name}.png"
        local_path = os.path.join(local_tmp_dir, filename)
        fig.savefig(local_path)
        plt.close(fig)

        # Upload to plot_log folder on Drive
        drive_path = os.path.join(output_dir_drive_path, filename)
        drive_path = os.path.normpath(drive_path)

        final_drive_path = f"/content/drive/MyDrive/{drive_path}"
        os.makedirs(os.path.dirname(final_drive_path), exist_ok=True)
        os.rename(local_path, final_drive_path)

        uploaded_file_ids.append(final_drive_path)

    return uploaded_file_ids

## Spartial View

In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

def generate_data_and_plots_for_all_folders(root_path):
    for folder_name in os.listdir(root_path):
        folder_path = os.path.join(root_path, folder_name)
        json_path = os.path.join(folder_path, 'raw_data.json')
        plot_log_path = os.path.join(folder_path, 'plot_log')
        os.makedirs(plot_log_path, exist_ok=True)

        if os.path.exists(json_path):
            print(f"📂 Processing: {folder_name}")
            data = read_json_data(json_path)
            #print(len(data))
            df = data_to_dataframe(data)
            #print(len(df))

            # Save CSV
            csv_path = os.path.join(folder_path, 'skeleton_data.csv')
            df.to_csv(csv_path, index=False)

            # Generate plots
            for _, row in df.iterrows():
                entry_id = row['EntryID']
                # view_configs = [
                #     ('view1', 20, 70),
                #     ('view2', -90, 90),
                #     ('view3', 40, 0)
                # ]

                view_configs = [
                    ('view1', 20, -60),
                    ('view2', -90, 90),
                    ('view3', 0, 20)
                ]


                for view_name, elev, azim in view_configs:
                    fig = plt.figure()
                    ax = fig.add_subplot(111, projection='3d')
                    ax.view_init(elev=elev, azim=azim)
                    plot_skeleton_on_ax(row, ax)
                    plt.tight_layout()
                    fig.savefig(os.path.join(plot_log_path, f"{entry_id}_{view_name}.png"))
                    plt.close(fig)

generate_data_and_plots_for_all_folders('/content/drive/MyDrive/VR_Archery')

📂 Processing: archery02
📂 Processing: archery01
📂 Processing: archery03


In [7]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

# Your existing functions should be defined already:
# - read_json_data()
# - data_to_dataframe()
# - plot_skeleton_on_ax()

def generate_missing_views_for_entry(root_path, folder_name, target_entry_id):
    folder_path = os.path.join(root_path, folder_name)
    json_path = os.path.join(folder_path, 'raw_data.json')
    plot_log_path = os.path.join(folder_path, 'plot_log')
    os.makedirs(plot_log_path, exist_ok=True)

    if os.path.exists(json_path):
        print(f"📂 Processing: {folder_name}")
        data = read_json_data(json_path)
        df = data_to_dataframe(data)

        # Find the row for the specific EntryID
        row = df[df["EntryID"] == target_entry_id]
        if row.empty:
            print(f"❌ EntryID {target_entry_id} not found in {folder_name}")
            return
        row = row.iloc[0]

        view_configs = [
            ('view1', 20, -60),
            ('view2', -90, 90)
        ]

        for view_name, elev, azim in view_configs:
            fig = plt.figure()
            ax = fig.add_subplot(111, projection='3d')
            ax.view_init(elev=elev, azim=azim)
            plot_skeleton_on_ax(row, ax)
            plt.tight_layout()
            save_path = os.path.join(plot_log_path, f"{target_entry_id}_{view_name}.png")
            fig.savefig(save_path)
            plt.close(fig)
            print(f"✅ Saved: {save_path}")

# Run for specific case
generate_missing_views_for_entry(
    root_path='/content/drive/MyDrive/VR_Archery',
    folder_name='archery02',
    target_entry_id='06262025083949682'
)

📂 Processing: archery02
✅ Saved: /content/drive/MyDrive/VR_Archery/archery02/plot_log/06262025083949682_view1.png
✅ Saved: /content/drive/MyDrive/VR_Archery/archery02/plot_log/06262025083949682_view2.png


## Interactive View

In [ ]:
from scipy.spatial.transform import Rotation as R

def quaternion_to_rotation_matrix(qx, qy, qz, qw):
    return R.from_quat([qx, qy, qz, qw]).as_matrix()

def plot_skeleton_interaction_on_ax(row, ax):
    import numpy as np

    # Joint groups
    bone_groups = {
        'spine': [
            ('joint_Pelvis', 'joint_HipMaster'),
            ('joint_HipMaster', 'joint_TorsoA'),
            ('joint_TorsoA', 'joint_TorsoB'),
            ('joint_TorsoB', 'joint_TorsoC'),
            ('joint_TorsoC', 'joint_Neck'),
            ('joint_Neck', 'joint_Head'),
        ],
        'left_arm': [
            ('joint_TorsoC', 'joint_ClavicleLT'),
            ('joint_ClavicleLT', 'joint_ShoulderLT'),
            ('joint_ShoulderLT', 'joint_ElbowLT'),
            ('joint_ElbowLT', 'joint_HandLT'),
        ],
        'right_arm': [
            ('joint_TorsoC', 'joint_ClavicleRT'),
            ('joint_ClavicleRT', 'joint_ShoulderRT'),
            ('joint_ShoulderRT', 'joint_ElbowRT'),
            ('joint_ElbowRT', 'joint_HandRT'),
        ],
        'left_leg': [
            ('joint_Pelvis', 'joint_HipLT'),
            ('joint_HipLT', 'joint_KneeLT'),
            ('joint_KneeLT', 'joint_FootLT'),
            ('joint_FootLT', 'joint_ToeLT'),
        ],
        'right_leg': [
            ('joint_Pelvis', 'joint_HipRT'),
            ('joint_HipRT', 'joint_KneeRT'),
            ('joint_KneeRT', 'joint_FootRT'),
            ('joint_FootRT', 'joint_ToeRT'),
        ]
    }

    limb_colors = {
        'spine': 'blue',
        'left_arm': 'cyan',
        'right_arm': 'fuchsia',
        'left_leg': 'red',
        'right_leg': 'lime'
    }

    # Parse joint coordinates
    joint_names = sorted(set(
        col[:-2] for col in row.index
        if col.startswith("joint_") and col.endswith(('_X', '_Y', '_Z')) and not col.startswith("joint_Char")
    ))
    joint_coords = {
        name: (float(row[f'{name}_X']), float(row[f'{name}_Y']), float(row[f'{name}_Z']))
        for name in joint_names
    }

    # Identify connected joints (used in bones)
    connected_joints = set(j for bone_list in bone_groups.values() for j1, j2 in bone_list for j in (j1, j2))

    # Plot joints
    for name, coord in joint_coords.items():
        size = 10 if name in connected_joints else 5
        ax.scatter(*coord, color='blue', s=size)

    # Plot bones
    for limb, bone_list in bone_groups.items():
        color = limb_colors[limb]
        for j1, j2 in bone_list:
            if j1 in joint_coords and j2 in joint_coords:
                ax.plot(
                    [joint_coords[j1][0], joint_coords[j2][0]],
                    [joint_coords[j1][1], joint_coords[j2][1]],
                    [joint_coords[j1][2], joint_coords[j2][2]],
                    color=color
                )

    # Plot targets as vertical bullseye-like circles in the Y-Z plane
    def plot_target(name, color, radius=0.5, num_rings=3):
        # cx, cy, cz = float(row[f'{name}_X']), float(row[f'{name}_Y']), float(row[f'{name}_Z'])
        # theta = np.linspace(0, 2 * np.pi, 200)

        # for i in range(num_rings, 0, -1):
        #     r = radius * (i / num_rings)
        #     y = cy + r * np.cos(theta)
        #     z = cz + r * np.sin(theta)
        #     x = np.full_like(y, cx)
        #     ax.plot(x, y, z, color=color, linewidth=2 if i == 1 else 1, alpha=0.8)

        # ax.scatter(cx, cy, cz, color=color, s=60, marker='x')

        #-----------------------------------
        # cx = float(row[f'{name}_X'])
        # cy = float(row[f'{name}_Y'])
        # cz = float(row[f'{name}_Z'])

        # theta = np.linspace(0, 2 * np.pi, 100)
        # y = cy + radius * np.cos(theta)
        # z = cz + radius * np.sin(theta)
        # x = np.full_like(theta, cx)

        # ax.plot(x, y, z, color=color, linewidth=2)
        # ax.scatter(cx, cy, cz, color=color, s=60, marker='x')

        # Center
        cx = float(row[f'{name}_X'])
        cy = float(row[f'{name}_Y'])
        cz = float(row[f'{name}_Z'])

        # Quaternion
        qx = float(row[f'{name}_RotX'])
        qy = float(row[f'{name}_RotY'])
        qz = float(row[f'{name}_RotZ'])
        qw = float(row[f'{name}_RotW'])

        # Get rotation matrix
        R_mat = quaternion_to_rotation_matrix(qx, qy, qz, qw)

        # Define circle in X-Y plane (local)
        theta = np.linspace(0, 2 * np.pi, 100)
        circle_pts = np.stack([
            radius * np.cos(theta),  # X
            radius * np.sin(theta),  # Y
            np.zeros_like(theta)     # Z = 0 (flat)
        ], axis=1)  # shape: (100, 3)

        # Rotate the circle to match target rotation
        rotated_circle = circle_pts @ R_mat.T  # shape: (100, 3)

        # Translate to target position
        x, y, z = rotated_circle[:, 0] + cx, rotated_circle[:, 1] + cy, rotated_circle[:, 2] + cz

        # Plot
        ax.plot(x, y, z, color=color, linewidth=2)
        ax.scatter(cx, cy, cz, color=color, s=60, marker='x')

        # print(f"{name} rotation quaternion: ({qx}, {qy}, {qz}, {qw})")
        # print(f"Rotation matrix:\n{R_mat}")



    plot_target('Target', 'orange', radius=1.5)
    plot_target('Target1', 'green', radius=1.5)
    plot_target('Target2', 'red', radius=1.5)

    # Arrow trajectory calculation
    pos = np.array([float(row['Arrow_X']), float(row['Arrow_Y']), float(row['Arrow_Z'])])

    qx = float(row['Arrow_RotX'] )
    qy = float(row['Arrow_RotY'])
    qz = float(row['Arrow_RotZ'])
    qw = float(row['Arrow_RotW'])

    # Forward vector from quaternion
    forward = np.array([
        2 * (qx * qz + qw * qy),
        2 * (qy * qz - qw * qx),
        1 - 2 * (qx**2 + qy**2)
    ])
    velocity = forward * 30 * 1.5
    gravity = np.array([0, -9.81, 0])
    times = np.arange(0, 4.1, 0.1)

    traj_points = [pos + velocity * t + 0.5 * gravity * t**2 for t in times]
    traj_points = np.array(traj_points)
    ax.plot(traj_points[:, 0], traj_points[:, 1], traj_points[:, 2], color='red', linestyle='--', label='Arrow Trajectory')

    # Labels
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")


    # Focus the view on the user + targets (exclude trajectory from limits)
    joint_xyz = np.array(list(joint_coords.values()))

    # Get target coordinates if they exist
    target_coords = []
    for target in ['Target', 'Target1', 'Target2']:
        try:
            tx = float(row[f'{target}_X'])
            ty = float(row[f'{target}_Y'])
            tz = float(row[f'{target}_Z'])
            target_coords.append([tx, ty, tz])
        except:
            continue

    # Combine skeleton and target coordinates
    focus_points = np.vstack([joint_xyz] + target_coords)

    # Set axis limits with padding
    padding = 0.3
    x_min, y_min, z_min = focus_points.min(axis=0) - padding
    x_max, y_max, z_max = focus_points.max(axis=0) + padding

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_zlim(z_min, z_max)

In [ ]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt

def generate_interaction_data_and_plots_for_all_folders(root_path):
    for folder_name in os.listdir(root_path):
        folder_path = os.path.join(root_path, folder_name)
        json_path = os.path.join(folder_path, 'raw_data.json')
        plot_log_path = os.path.join(folder_path, 'plot_log')
        os.makedirs(plot_log_path, exist_ok=True)

        if os.path.exists(json_path):
            print(f"📂 Processing: {folder_name}")
            data = read_json_data(json_path)
            #print(len(data))
            df = data_to_dataframe(data)
            #print(len(df))

            # Save CSV
            csv_path = os.path.join(folder_path, 'skeleton_data.csv')
            df.to_csv(csv_path, index=False)

            # Generate plots
            for _, row in df.iterrows():
                entry_id = row['EntryID']
                # view_configs = [
                #     ('view1', 20, 70),
                #     ('view2', -90, 90),
                #     ('view3', 40, 0)
                # ]

                view_configs = [
                    ('view1', 20, -60),
                    ('view2', -90, 90),
                    ('view3', 0, 20)
                ]


                for view_name, elev, azim in view_configs:
                    fig = plt.figure()
                    ax = fig.add_subplot(111, projection='3d')
                    ax.view_init(elev=elev, azim=azim)
                    plot_skeleton_interaction_on_ax(row, ax)
                    plt.tight_layout()
                    fig.savefig(os.path.join(plot_log_path, f"{entry_id}_interaction_{view_name}.png"))
                    plt.close(fig)

generate_interaction_data_and_plots_for_all_folders('/content/drive/MyDrive/VR_Archery')

📂 Processing: archery02
📂 Processing: archery01
📂 Processing: archery03


In [ ]:
pip install gspread oauth2client gspread_dataframe

In [2]:
import os
import pandas as pd
import gspread
from google.colab import auth
from gspread_dataframe import set_with_dataframe
from google.auth import default
from googleapiclient.discovery import build

# Authenticate and set up services
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive_service = build('drive', 'v3', credentials=creds)

# --------- Helper Functions ---------
def get_folder_id_by_path(path_list):
    """Walk through the path to get the folder ID."""
    parent_id = 'root'
    for name in path_list:
        query = f"mimeType='application/vnd.google-apps.folder' and trashed = false and name='{name}' and '{parent_id}' in parents"
        results = drive_service.files().list(q=query, spaces='drive', fields="files(id, name)", pageSize=10).execute()
        items = results.get('files', [])
        if not items:
            raise FileNotFoundError(f"Folder '{name}' not found under parent ID '{parent_id}'")
        parent_id = items[0]['id']
    return parent_id

# def get_public_image_link(file_name, folder_id):
#     """Find the file in Drive, make it public, and return an IMAGE formula for Sheets."""
#     query = f"name='{file_name}' and '{folder_id}' in parents and trashed = false"
#     results = drive_service.files().list(q=query, fields="files(id, name)").execute()
#     items = results.get('files', [])
#     if not items:
#         return ""  # Skip if image not found

#     file_id = items[0]['id']

#     # Make it public
#     try:
#         drive_service.permissions().create(
#             fileId=file_id,
#             body={"role": "reader", "type": "anyone"},
#             fields="id"
#         ).execute()
#     except:
#         pass  # Permission might already exist

#     # Create direct image link
#     public_url = f"https://drive.google.com/uc?export=download&id={file_id}"
#     return f'=IMAGE("{public_url}")'

def get_public_image_link(file_name, folder_id):
    """Find the file in Drive, make it public, and return an IMAGE formula for Sheets."""
    query = f"name='{file_name}' and '{folder_id}' in parents and trashed = false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    items = results.get('files', [])
    if not items:
        return ""  # Skip if image not found

    file_id = items[0]['id']

    # Make it public
    try:
        drive_service.permissions().create(
            fileId=file_id,
            body={"role": "reader", "type": "anyone"},
            fields="id"
        ).execute()
    except:
        pass  # Permission might already exist

    # Return formula for Sheets
    public_url = f"https://drive.google.com/uc?export=view&id={file_id}"
    return f'=IMAGE("{public_url}")'



# --------- Base Setup ---------
base_path = "/content/drive/MyDrive/VR_Archery"
folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]

# --------- Main Loop for Each Folder ---------
for folder in folders:
    print(f"📁 Processing folder: {folder}")
    try:
        # Get Google Drive folder IDs for plot_log and image_log
        plot_folder_id = get_folder_id_by_path(["VR_Archery", folder, "plot_log"])
        image_folder_id = get_folder_id_by_path(["VR_Archery", folder, "image_log"])
    except FileNotFoundError as e:
        print(f"⚠️ Skipping {folder}: {e}")
        continue

    # Get EntryIDs from plot_log view1 files
    local_plot_path = os.path.join(base_path, folder, "plot_log")
    entry_ids = sorted(set(
        fname.split('_')[0]
        for fname in os.listdir(local_plot_path)
        if fname.endswith('.png') and '_view1' in fname
    ))

    # Build dataframe
    records = []
    for entry_id in entry_ids:
        record = {
            "EntryID": entry_id,
            "view1": get_public_image_link(f"{entry_id}_view1.png", plot_folder_id),
            "view2": get_public_image_link(f"{entry_id}_view2.png", plot_folder_id),
            "view3": get_public_image_link(f"{entry_id}_view3.png", plot_folder_id),
            "Image": get_public_image_link(f"{entry_id}.png", image_folder_id),
            "Spatial 1": "", "Spatial 2": "", "Spatial": "",
            "interaction_view1": get_public_image_link(f"{entry_id}_interaction_view1.png", plot_folder_id),
            "interaction_view2": get_public_image_link(f"{entry_id}_interaction_view2.png", plot_folder_id),
            "interaction_view3": get_public_image_link(f"{entry_id}_interaction_view3.png", plot_folder_id),
            "image": get_public_image_link(f"{entry_id}.png", image_folder_id),
            "Temporal1": "", "Temporal2": "", "Temporal": ""
        }
        records.append(record)

    df = pd.DataFrame(records)

    # Create Google Sheet
    sheet_title = f"{folder}_LabelSheet"
    try:
        sh = gc.create(sheet_title)
        worksheet = sh.get_worksheet(0)
        set_with_dataframe(worksheet, df)
        print(f"✅ Created sheet: {sheet_title}")
    except Exception as e:
        print(f"❌ Failed to create sheet for {folder}: {e}")

📁 Processing folder: archery02
✅ Created sheet: archery02_LabelSheet
📁 Processing folder: archery01
✅ Created sheet: archery01_LabelSheet
📁 Processing folder: archery03
✅ Created sheet: archery03_LabelSheet
